# Suspicious Certutil Command
Detects a suspicious Microsoft certutil execution with sub commands like 'decode' sub command, which is sometimes used to decode malicious code with the built-in certutil utility

## Rule Content
```
- title: Suspicious Certutil Command
  id: e011a729-98a6-4139-b5c4-bf6f6dd8239a
  status: experimental
  description: Detects a suspicious Microsoft certutil execution with sub commands
    like 'decode' sub command, which is sometimes used to decode malicious code with
    the built-in certutil utility
  author: Florian Roth, juju4, keepwatch
  modified: 2019/01/22
  references:
  - https://twitter.com/JohnLaTwC/status/835149808817991680
  - https://twitter.com/subTee/status/888102593838362624
  - https://twitter.com/subTee/status/888071631528235010
  - https://blogs.technet.microsoft.com/pki/2006/11/30/basic-crl-checking-with-certutil/
  - https://www.trustedsec.com/2017/07/new-tool-release-nps_payload/
  - https://twitter.com/egre55/status/1087685529016193025
  - https://lolbas-project.github.io/lolbas/Binaries/Certutil/
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      CommandLine:
      - '* -decode *'
      - '* /decode *'
      - '* -decodehex *'
      - '* /decodehex *'
      - '* -urlcache *'
      - '* /urlcache *'
      - '* -verifyctl *'
      - '* /verifyctl *'
      - '* -encode *'
      - '* /encode *'
      - '*certutil* -URL*'
      - '*certutil* /URL*'
      - '*certutil* -ping*'
      - '*certutil* /ping*'
    condition: selection
  fields:
  - CommandLine
  - ParentCommandLine
  tags:
  - attack.defense_evasion
  - attack.t1140
  - attack.t1105
  - attack.s0189
  - attack.g0007
  falsepositives:
  - False positives depend on scripts and administrative tools used in the monitored
    environment
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='process_command_line.keyword:(*\ \-decode\ * OR *\ \/decode\ * OR *\ \-decodehex\ * OR *\ \/decodehex\ * OR *\ \-urlcache\ * OR *\ \/urlcache\ * OR *\ \-verifyctl\ * OR *\ \/verifyctl\ * OR *\ \-encode\ * OR *\ \/encode\ * OR *certutil*\ \-URL* OR *certutil*\ \/URL* OR *certutil*\ \-ping* OR *certutil*\ \/ping*)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()